__Autor: Christian Camilo Urcuqui López__

__Date: 24 February 2019__

# Reasoning

It is an attribute of human being. The capacity of reasoning allows any person to remember information related to events and make new knowledge about things that we are not lived yet.

This capacity allows us to classify objects and situations, through this we can recognize them and make decisions about them. For example, if someone shows us a computer we can identify it through its features, we can do this with things that we knew previously, for example:

+ It has a mouse
+ It has a monitor
+ It has a keyboard
+ Etc.

## Expert systems

<img src="https://www.igcseict.info/theory/7_2/expert/files/stacks_image_5738.png" />

Since the first artificial intelligence works have tried to develop computers with reasoning capacities like humans. The first systems were called knowledge-based system (KBS) which don't reason but they are specially designed to resolve a specific complex problem through the application of logic rules (it is called the knowledge domine). 

These solutions have two main components, they are:

+ The Knowledge Base

This focuses to save the information about the domain to make conclusions about the inputs. 
+ The Inference Engine

It is the reasoning process about the solution which searches information in the knowledge edge and it relates them to get a coherent conclusion related to the problem approached.

In a __Expert System__ the _inference engine_ is based in rules and a base of acts or memory of works. 

An act has information about a domain, and it allows us to associate acts with other new acts, like the next rule

antecedent(x) $\to$ consequent(x) 

In the inference engine we can find two search strategies, the most used is called _forward chaining_ and another is named _backward chaining_. In the forward chaining, the expert system makes inferences according to the antecedents of the rules.


This notebook is divided in the next sections to explore the application of expert systems in Python:

+ [Pyknow](#PyKnow)
+ [Conditional Elements (CE): composing Patterns Together](#Conditional-Elements-(CE):-composing-Patterns-Together)
+ [Field Constraints: FC for sort](#Field-Constraints:-FC-for-sort)
+ [Composing FCs: &, | and ~](#Composing-FCs:-&,-|-and-~)
+ [Variable Binding: The << Operator](#Variable-Binding:-The-<<-Operator)
+ [Examples](#Examples)
    + [Trafic Lights](#Trafic-Lights)
    + [Chat](#Chat)
    + [Rock–paper–scissors](#Rock–paper–scissors)

## PyKnow

let's see some frameworks which allows us to build some components of an expert system. Based in CLIPS (http://www.clipsrules.net/) PyKnow is going to give us the ways to apply this area. 

Let's install the library

```
pip install pyknow
```


In [1]:
from pyknow import *

__Fact__ is a basic unit of PyKnow which are used by the system to reason about the problem. 

__Rule__ is a callable, decorated with _Rule_. Rules have two components, LHS (left-hand-side) and RHS (right-hand-side).

+ The LHS describes (using patterns) the conditions on which the rule _n_ should be executed.
+ The RHS is the set of actions to perform when the rule is fired

```
@Rule(<pattern_1>,
<pattern_2>,
...
<pattern_n>)
def _():
pass
```
"It behaves like AND by default"


For a Fact to match a Pattern, all pattern restrictions must be True when the Fact is evaluated against it. 

<img src="https://pyknow.readthedocs.io/en/stable/_images/graphviz-ce8d678ec7b2aa565c8b008562d17fd2aac6e97a.png" />

In [4]:
f = Fact(a=1, b=2) # order is arbitraty
print(f['a'])

1


We can make a Fact without keys (only values), and Fact will create a numeric index for each value

In [5]:
f = Fact('x', 'y', 'z')
print(f[1])

y


Another think of Fact is the integration of different types of variables 

In [7]:
f = Fact('x', 'y', 'z', a=1, b=2)
print(f[0])
print(f['a'])

x
1


We can subclass Fact to express different kinds of data or extend it with our functionalities 

In [8]:
class Alert(Fact):
    pass
class Status(Fact):
    pass
f1 = Alert(color='red')
f2 = Status(state='critical')
print(f1['color'])
print(f2['state'])

red
critical


Most of the time expert systems needs a set of facts to be present for the system to work. This is the purporse of the DefFacts decorator. So, all the DefFacts inside a KnowledgeEngine will be called every time the reset method is executed. 

In [9]:
@DefFacts()
def needed_data():
    yield Fact(best_color='red')
    yield Fact(best_body='medium')
    yield Fact(best_sweetness='dry')    

In [10]:
# Rules
class MyFact(Fact):
    pass

@Rule(MyFact()) # This is the LHS
def matchWithEveryMYFact():
    # This is the RHS
    pass

For example, let's apply the next rule, math with every Fact which:
+ f[0] == 'animal'
+ f['family'] == 'felinae'

In [12]:
class MyFact(Fact):
    pass
@Rule(MyFact('animal', family='felinae'))
def match_with_cats():
    print("Meow!")

Another example, the user is a priviliged one and we are not dropping privileges.

In [15]:
class User(Fact):
    pass
@Rule(User('admin')|User('root'))
def the_user_has_power():
    enable_superpowers()

### Conditional Elements (CE): composing Patterns Together

__AND__

.This creates a composed pattern containing all Facts passed as arguments. All of the passed patterns must match for the composed pattern to match.

In [3]:
# Match if two facts are declared, one martching Fact(1) and other matching Fact(2)
@Rule(AND(Fact(1),Fact(2)))
def _():
    pass

__OR__

It creates a composed pattern in which any of the given pattern will make the rule match

In [4]:
# Math if a fact matching Fact(1) exists and/or a fact matching Fact(2)
@Rule(OR(Fact(1), Fact(2)))
def _():
    pass

__NOT__

This elements matches if the given pattern does not match with any fact or combination of facts. Therefore this element matches the absence of the given pattern

In [5]:
# Match if not fact with Fact(1)
@Rule(NOT(Fact(1)))
def _():
    pass

__TEST__

It checks the reveived callable against the current binded values. If the execution returns True the evaluation will continue and stops otherwise

In [23]:
# Match for all numbers a, b, c where a > b > c
class Number(Fact):
    pass
@Rule(Number(MATCH.a),
    Number(MATCH.b),
    TEST(lambda a, b: a > b),
    Number(MATCH.c),
    TEST(lambda b, c: b > c))
def _(a, b, c):
    pass

__EXISTS__

EXISTS receives a pattern and matches if one or more facts matches this pattern. This will match only once while one ore more matching facts exists and will stop matching when there is no matching facts.

In [9]:
# Match once when one or more Color exists
class Color(Fact):
    pass
@Rule(EXISTS(Color()))
def _():
    pass

__FORALL__

This conditional element provides a mechanism for determining if a group of specified Conditional Element is satisfied for every occurrence of another specified CE

In [10]:
# Match once when all CE are satisfied
class Student(Fact):
    pass
class Reading(Fact):
    pass
class Writing(Fact):
    pass
class Arithmetic(Fact):
    pass
@Rule(FORALL(Student(W('name')),
            Reading(W('name')),
            Writing(W('name')),
            Arithmetic(W('name'))))

def all_students_passed():
    pass

## Field Constraints: FC for sort

__L (Literal Field Constraint)__

It allows the program to perform an exact match with the given value. The matching is donde using the equality operator ==.

In [11]:
# Match if the first element is exactly 3
@Rule(Fact(L(3)))
def _():
    pass

__W (Wildcard Field Constraint)__

This element matches with any value.

In [13]:
# match if some fact is declared with the key mykey
@Rule(Fact(mykey=W()))
def _():
      pass

__P (Predicate Field Constraint):__

The match of this element is the result of applying the given callable to the fact-extracted value. If the callable returns True the FC will match, in other case the FC will not match. 

In [14]:
# match is some fact is declared whose first parameter is an instance of int
@Rule(Fact(P(lambda x: isinstance(x, int))))
def _():
    pass

## Composing FCs: &, | and ~

__ANDFC() - &__

The composed FC matches if all the given FC match.


In [21]:
# Match if key x of Point is a value between 0 and 255
@Rule(Fact(x=P(lambda x: x >= 0) & P(lambda x: x <= 255)))
def _():
    pass

__ORFC() - |__

The composed FC matches if any of the given FC matches.

In [24]:
@Rule(Fact(name=L('Alice') | L('Bob')))
def _():
    pass

__NOTFC() - ~__

This composed FC negates the given FC, reversing the logic FC. 

In [25]:
# Match if name is not Charlie
@Rule(Fact(name=~L('Charlie')))
def _():
    pass

## Variable Binding: The << Operator

Any pattern and some FC's can be binded to a name using the << operator.

In [26]:
# The first value of the matching fact will be blinded to the name value and passed to the function when fired
@Rule(Fact('value' << W()))
def _(value):
    pass

This is deprecated since version 1.2.0, for the last case it is best to use _Match_.

In [28]:
@Rule('f1' << Fact())
def _(f1):
    pass

Deprecated since version 1.2.0: Use _AS_ object instead

### MATCH Object

This allows us to generate more readable name bindings.

In [29]:
@Rule(Fact(MATCH.myvalue))
def _(myvalue):
    pass

The last example is exactly the same as:

```
@Rule(Fact('myvalue' << W()))
def _(myvalue):
    pass
```

### AS Object

The AS object like the MATCH object is syntatic sugar for generating bindable names. In this case any attribute requested to the AS object will return a string with the same name.

In [30]:
@Rule(AS.myfact << Fact(W()))
def _(myfact):
    pass

The last example is exactly the same as:

```
@Rule("myfact" << Fact(W()))
def _(myfact):
    pass
```

## Examples

### Trafic Lights

In the next example we can implement our first hello world of expert systems in Python, look how we made the rules and how we defined their results. 

In [1]:
from random import choice
from pyknow import *

class Light(Fact):
    """Info about the traffic light."""
    pass


class RobotCrossStreet(KnowledgeEngine):
    @Rule(Light(color='green'))
    def green_light(self):
        print("Walk")

    @Rule(Light(color='red'))
    def red_light(self):
        print("Don't walk")

    @Rule(AS.light << Light(color=L('yellow') | L('blinking-yellow')))
    def cautious(self, light):
        print("Be cautious because light is", light["color"])

The knowledgeEngine is the place where all the magic happens, the first step is to make a subclass of it and use Rule to decorate its methods. The next step is to instantiate it and run it. 

This is the usual process to execute a KnowledgeEngine.

+ The class must be instantiated, of course.
+ The reset method must be called:
    + This declares the special fact InitialFact. Necessary for some rules to work properly.
    + Declare all facts yielded by the methods decorated with @DefFacts.
+ The run method must be called. This starts the cycle of execution.

In [2]:
engine = RobotCrossStreet()
engine.reset()
engine.declare(Light(color=choice(['green', 'yellow', 'blinking-yellow', 'red'])))
engine.run()

Be cautious because light is blinking-yellow


### Chat

In [3]:
from random import choice
from pyknow import *

class Greetings(KnowledgeEngine):
    # Most of the time expert systems needs a set of facts to be present 
    # for the system to work. This is the purpose of the DefFacts decorator
    @DefFacts()
    def _initial_action(self):
        yield Fact(action="greet")

    @Rule(Fact(action='greet'),
          NOT(Fact(name=W())))
    def ask_name(self):
        self.declare(Fact(name=input("What's your name? ")))

    @Rule(Fact(action='greet'),
          NOT(Fact(location=W())))
    def ask_location(self):
        self.declare(Fact(location=input("Where are you? ")))

    @Rule(Fact(action='greet'),
          Fact(name=MATCH.name),
          Fact(location=MATCH.location))
    def greet(self, name, location):
        print("Hi %s! How is the weather in %s?" % (name, location))

engine = Greetings()
engine.reset()  # Prepare the engine for the execution.
engine.run()  # Run it!

Where are you? icesi
What's your name? kuky
Hi kuky! How is the weather in icesi?


### Rock–paper–scissors

Let's make a Rock–paper–scissors game 

In [4]:
from pyknow import *
import random

NERD = True

In [5]:
class WinTotals(Fact):
    human = Field(int, default=0)
    computer = Field(int, default=0)
    ties = Field(int, default=0)
    
class Results(Fact):
    winner = Field(str, mandatory=True)
    loser = Field(str, mandatory=True)
    why = Field(str, mandatory=True)
    
class ValidAnswer(Fact):
    answer = Field(str, mandatory=True)
    key = Field(str, mandatory=True)

class Action(Fact):
    pass

class HumanChoice(Fact):
    pass

class ComputerChoice(Fact):
    pass

class RockPaperScissors(KnowledgeEngine):
    def yes_or_no(self, question):
        return input(question).upper().startswith('Y')
    
    @DefFacts()
    def game_rules(self, is_nerd=False):
        """Declare game rules and valid input keys for the user."""
        self.valid_answers = dict()
        
        yield Results(winner='rock', loser='scissors', why='Rock smashes scissors')
        yield Results(winner='paper', loser='rock', why='Paper covers rock')
        yield Results(winner='scissors', loser='paper', why='Scissors cut paper')
        yield ValidAnswer(answer='rock', key='r')
        yield ValidAnswer(answer='paper', key='p')
        yield ValidAnswer(answer='scissors', key='s')
        
        if is_nerd:
            yield Results(winner='rock', loser='lizard', why='Rock crushes lizard')
            yield Results(winner='spock', loser='rock', why='Spock vaporizes rock')
            yield Results(winner='spock', loser='scissors', why='Spock smashes scissors')
            yield Results(winner='paper', loser='spock', why='Paper disproves Spock')
            yield Results(winner='scissors', loser='lizard', why='Scissors decapitates lizard')
            yield Results(winner='lizard', loser='paper', why='Lizard eats paper')
            yield Results(winner='lizard', loser='spock', why='Lizard poisons Spock')
            yield ValidAnswer(answer='spock', key='k')
            yield ValidAnswer(answer='lizard', key='l')
            
    @Rule()
    def startup(self):
        print("Lets play a game!")
        print("You choose rock, paper, or scissors,")
        print("and I'll do the same.")
        self.declare(WinTotals(human=0, computer=0, ties=0))
        self.declare(Action('get-human-move'))
        
    @Rule(NOT(Action()),
          ValidAnswer(answer=MATCH.answer,
                      key=MATCH.key))
    def store_valid_answers(self, answer, key):
        self.valid_answers[key] = answer
        
    #
    # HUMAN MOVE RULES
    #
    @Rule(Action('get-human-move'))
    def get_human_move(self):
        question = ", ".join(
            "{name} ({key})".format(
                name=a[1].title(), key=a[0].upper())
            for a in self.valid_answers.items()) + '? '
        res = input(question).lower()
        self.declare(HumanChoice(res))
    
    @Rule(AS.f1 << HumanChoice(MATCH.choice),
          ValidAnswer(answer=MATCH.answer,
                      key=MATCH.choice),
          AS.f2 << Action('get-human-move'))
    def good_human_move(self, f1, f2, answer):
        self.retract(f1)
        self.retract(f2)
        self.declare(HumanChoice(answer))
        self.declare(Action('get-computer-move'))
    
    @Rule(AS.f1 << HumanChoice(MATCH.choice),
          NOT(ValidAnswer(key=MATCH.choice)),
          AS.f2 << Action('get-human-move'))
    def bad_human_move(self, f1, f2, choice):
        print("Sorry %s is not a valid answer" % choice)
        self.retract(f1)
        self.retract(f2)
        self.declare(Action('get-human-move'))
    
    #
    # COMPUTER MOVE RULES
    #
    @Rule(AS.f1 << Action('get-computer-move'))
    def get_computer_move(self, f1):
        choice = random.choice(list(self.valid_answers.values()))
        self.retract(f1)
        self.declare(ComputerChoice(choice))
        self.declare(Action('determine-results'))

    #
    # WIN DETERMINATION RULES
    #
    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.hc),
          AS.w << WinTotals(computer=MATCH.cw),
          Results(winner=MATCH.cc,
                  loser=MATCH.hc,
                  why=MATCH.explanation))
    def computer_wins(self, f1, f2, f3, w, cw, explanation):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, computer=cw + 1)
        print("Computer wins!", explanation)
        self.declare(Action('determine-play-again'))
        
    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.hc),
          'w' << WinTotals(human=MATCH.hw),
          Results(winner=MATCH.hc,
                  loser=MATCH.cc,
                  why=MATCH.explanation))
    def humans_wins(self, f1, f2, f3, w, hw, explanation):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, human=hw + 1)
        print("You win!", explanation)
        self.declare(Action('determine-play-again'))
        
    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.cc),
          AS.w << WinTotals(ties=MATCH.nt))
    def tie(self, f1, f2, f3, w, nt):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, ties=nt + 1)
        print("Tie! Ha-ha!")
        self.declare(Action('determine-play-again'))
    
    #
    # PLAY AGAIN RULE
    #
    @Rule(AS.f1 << Action('determine-play-again'),
          WinTotals(computer=MATCH.ct,
                    human=MATCH.ht,
                    ties=MATCH.tt))
    def play_again(self, f1, ct, ht, tt):
        self.retract(f1)
        if not self.yes_or_no("Play again?"):
            print("You won", ht, "game(s).")
            print("Computer won", ct, "game(s).")
            print("We tied", tt, "game(s).")
            self.halt()
        else:
            self.declare(Action('get-human-move'))
            
rps = RockPaperScissors()
rps.reset()
rps.run()

Lets play a game!
You choose rock, paper, or scissors,
and I'll do the same.
Scissors (S), Paper (P), Rock (R)? r
You win! Rock smashes scissors
Play again?s
You won 1 game(s).
Computer won 0 game(s).
We tied 0 game(s).


## Taller

Se propone como entregable de esta unidad la implementación de las reglas del proyecto "SISTEMA PARA UTILIZACIÓN EFICIENTE DE LOS RECURSOS DE FERTILIZACIÓN EN CULTIVOS DE CAÑA DE AZÚCAR EN EL VALLE GEOGRÁFICO DEL RIO CAUCA", proyecto realizado por los estudiantes y profesores: Álvaro Pachón De La Cruz, Gonzalo Llano Ramírez, Luis Eduardo Múnera, Camilo Barrios Pérez, Claudia Lubo, Julián Borrero, y Gonzalo Calderón. 

En el presente proyecto se han desarrollado distintos componentes que permiten ejecutar el diagnóstico del suelo, la propuesta de recomendaciones con base al diagnóstico y a la implementación de la fertilización.

La propuesta implementa un sistema experto con alrededor de 346 reglas que hacen parte de los componentes mencionados. Con el fin que ustedes puedan ver la aplicación de un sistema experto en Python, se le ha encomendado implementar las primeras __48__ reglas que permiten realizar el diagnostico con base a tres variables: 
+ Ph
+ CE (conductividad eléctrica)
+ Textura del suelo

Las reglas las puede encontrar en el siguiente repositorio:
https://github.com/kmilichus/AgroTIC/blob/master/ExpertoFertilizacion/src/main/resources/reglas.drl


## References

+ https://media.readthedocs.org/pdf/pyknow/latest/pyknow.pdf
+ https://es.slideshare.net/ahmadhussein45/expert-system-with-python-2

# 48 REGLAS - BEYCKER ÁGREDO - NICOLÁS TABORDA

In [2]:
from pyknow import *

class AnalisisDeSuelo(Fact):
    pass
#rule 0
@Rule(AnalisisDeSuelo(ph=P(lambda ph: ph>=7.2)))
def metodo():
    
        print("\n\n\n")
        print("==> ph: ALCALINO")
        
#rule 1       
@Rule(AnalisisDeSuelo(ph=P(lambda ph: 6.8<ph<7.2)))
def metodo():
    
        print("\n\n\n")
        print("==> ph: LIGERAMENTE ALCALINO")

#rule 2  
@Rule(AnalisisDeSuelo(ph=P(lambda ph: ph >=6.2) & P(lambda ph: ph<=6.8)))
def metodo():
    
        print("\n\n\n")
        print("==> ph: NEUTRO")
        
#rule 3       
@Rule(AnalisisDeSuelo(ph=P(lambda ph: 5.6<ph<6.2)))
def metodo():
    
        print("\n\n\n")
        print("==> ph: LIGERAMENTE ALCALINO")
        
#rule 4     
@Rule(AnalisisDeSuelo((lambda ph: ph<=5.6)))
def metodo():
    
        print("\n\n\n")
        print("==> ph: ACIDO")

#rule 5      
@Rule(AnalisisDeSuelo(CE=P(lambda CE: CE < 0.8)))
def metodo():
    
        print("\n\n\n")
        print("==> conductividadElectrica: BAJA")
        
#rule 6   
@Rule(AnalisisDeSuelo(CE=P(lambda CE: CE >= 0.8)))
def metodo():
    
        print("\n\n\n")
        print("==> conductividadElectrica: ALTA")
        
#rule 7 
@Rule(AnalisisDeSuelo((lambda PH:PH==ALCALINO)))
def metodo():
    
        print("\n\n\n")
        print("==> ExtractoSoluble: True")
        
#rule 8     
@Rule(AnalisisDeSuelo(PH=P(lambda PH:PH=="LIGERAMENTE ALCALINO")))
def metodo():
    
        print("\n\n\n")
        print("==> ExtractoSoluble: True")
        
#rule 9
@Rule(AnalisisDeSuelo(arcilla=P(lambda arcilla:arcilla>=40.0) & P(lambda PH:PH=="ALCALINO") &  P(lambda CE:CE=="ALTA") ))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Limitaciones de movimientos de agua")
        print("     |-(2) Baja difusion de oxigeno y flujo de gases")
        print("     |-(3) Baja mineralizacion de MO (Baja actividad microbiologica)")
        print("     |-(4) Acumulacion de iones alcalinoterreos    ")
    

#rule 10
@Rule(AnalisisDeSuelo(arena=P(lambda arena:arena>=50.0) & P(lambda PH:PH=="ALCALINO")& P(lambda CE:CE=="ALTA")))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Revisar las mediciones realizadas")

#rule 11     
@Rule(AnalisisDeSuelo(arena=P(lambda arena:arena>=50.0) & P(lambda PH:PH=="ALCALINO")& P(lambda CE:CE=="ALTA")))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Coloraciones grises suelo (Glaizeado)");
        print("     |-(2) Suelo Hidromorfico");    
        print("     |-(3) Limitaciones fisicas temporales");
        print("     |-(4) Baja difusion de Oxigeno y flujo de gases");

#rule 12     
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo <= 40.0) &  P(lambda arena:arena <= 40.0) & P(lambda arcilla:arcilla <= 40.0) & P(lambda PH:PH == "ALCALINO") & P(lambda CE:CE == "ALTA")))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Baja disponibilidad de Fosforo (Precipitación)")
        print("     |-(2) Baja disponibilidad de Calcio")

        
#rule 13     
@Rule(AnalisisDeSuelo(arcilla=P(lambda arcilla:arcilla >= 40.0) & P(lambda PH:PH == "ALCALINO") & P(lambda CE:CE == "BAJA")))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Limitaciones de movimiento de agua");
        print("     |-(2) Baja difusion de Oxigeno y flujo de gases");    
        print("     |-(3) Baja mineralizacion de MO (Baja actvidad microbiologica");
        print("     |-(4) Acumulacion de iones alcalinoterreos	");
        print("     |-(5) Baja disponibilidad de elementos menores ");


        
#rule 14 
@Rule(AnalisisDeSuelo(arena=P(lambda arena:arena >= 50.0) & P(lambda PH:PH == "ALCALINO") & P(lambda CE:CE == "BAJA")))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Revisar las mediciones realizadas.");
        
            
        
#rule 15    
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo >= 45.0) & P(lambda PH:PH == "ALCALINO") & P(lambda CE:CE == "BAJA")))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Coloraciones grises suelo (Glaizeado)");
        print("     |-(2) Suelo Hidromorfico");    
        print("     |-(3) Limitaciones fisicas temporales");
        print("     |-(4) Baja difusion de Oxigeno y flujo de gases");
        print("     |-(5) Baja disponibilidad de elementos menores ");

#rule 16  
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo <= 40.0) &  P(lambda arena:arena <= 40.0) & P(lambda PH:PH == "ALCALINO") & P(lambda CE:CE == "ALTA")))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Baja disponibilidad de Fosforo (Precipitación)");
        print("     |-(2) Baja disponibilidad de Calcio");   
        print("     |-(3) Baja disponibilidad de elementos menores ")

#rule 17     
@Rule(AnalisisDeSuelo(arcilla=P(lambda arcilla:arcilla >= 40.0) & P(lambda PH:PH == "LIGERAMENTE ALCALINO") & P(lambda CE:CE == "ALTA")))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Limitaciones de movimiento de agua");
        print("     |-(2) Baja difusion de Oxigeno y flujo de gases");    
        print("     |-(3) Baja mineralizacion de MO (Baja actvidad microbiologica");
        print("     |-(4) Acumulacion de iones alcalinoterreos	");
        print("     |-(5) Alta saturación de calcio");
        print("     |-(6) Salinidad en el suelo");
        print("     |-(7) Baja disponibilidad de Fosforo (Precipitación)");

#rule 18    
@Rule(AnalisisDeSuelo(areba=P(lambda arena:arena >= 50.0) & P(lambda PH:PH == "LIGERAMENTE ALCALINO") & P(lambda CE:CE == "ALTA")))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Revisar las mediciones realizadas.");

        
#rule 19     
@Rule(AnalisisDeSuelo(arcilla=P(lambda arcilla:arcilla >= 45.0) & P(lambda PH:PH == "LIGERAMENTE ALCALINO") & P(lambda CE:CE == "ALTA")))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Coloraciones grises suelo (Glaizeado)")
        print("     |-(2) Suelo Hidromorfico")  
        print("     |-(3) Limitaciones fisicas temporales")
        print("     |-(4) Baja difusion de Oxigeno y flujo de gases")
        
#rule 20
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo <= 40.0) & P(lambda arena:arena <= 40.0) & P(lambda PH:PH == "LIGERAMENTE ALCALINO")& P(lambda CE:CE == "ALTA")))
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Baja disponibilidad de Fosforo (Precipitación)")
        print("     |-(2) Baja disponibilidad de Calcio") 
        
#rule 21
@Rule(AnalisisDeSuelo(arcilla=P(lambda arcilla:arcilla >= 40.0) & P(lambda PH:PH <= "LIGERAMENTE ALCALINO") & P(lambda CE:CE == "BAJA")))
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Limitaciones de movimiento de agua")
        print("     |-(2) Baja difusion de Oxigeno y flujo de gases")    
        print("     |-(3) Baja mineralizacion de MO (Baja actvidad microbiologica")
        print("     |-(4) Acumulacion de iones alcalinoterreos")
        print("     |-(5) Baja disponibilidad de elementos menores ")

#rule 22
@Rule(AnalisisDeSuelo( arena=P(lambda arena:arena >= 50.0) & P(lambda PH:PH == "LIGERAMENTE ALCALINO") & P(lambda CE:CE == BAJA)))               
def metodo():
    
        print("\n\n\n")
        print("     |-(1) Revisar las mediciones realizadas.");
       
    
#rule 23
@Rule(AnalisisDeSuelo( limo=P(lambda limo:limo >= 45.0) & P(lambda PH:PH == "LIGERAMENTE ALCALINO") & P(lambda CE:CE== BAJA)))             
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Coloraciones grises suelo (Glaizeado)");
        print("     |-(2) Suelo Hidromorfico");    
        print("     |-(3) Limitaciones fisicas temporales");
        print("     |-(4) Baja difusion de Oxigeno y flujo de gases");
        print("     |-(5) Baja disponibilidad de elementos menores ");
        
#rule 24
@Rule(AnalisisDeSuelo( limo=P(lambda limo:limo <= 40.0) &  P(lambda arena:arena <= 40.0) & P(lambda arcilla:arcilla <= 40.0) & P(lambda PH:PH == "LIGERAMENTE ALCALINO") & P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Baja disponibilidad de Fosforo (Precipitación)");
        print("     |-(2) Baja disponibilidad de Calcio");   
        print("     |-(3) Baja disponibilidad de elementos menores "); 
        
        

#rule 25
@Rule(AnalisisDeSuelo( arcilla=P(lambda arcilla:arcilla >= 40.0) & P(lambda PH:PH == NEUTRO) & P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Baja mineralizacion de MO (Baja actvidad microbiologica)	");
        print("     |-(2) Baja difusion de Oxigeno y flujo de gases");    
        print("     |-(3) Salinidad en el suelo");
        
        
        
#rule 26
@Rule(AnalisisDeSuelo( arena=P(lambda arena:arena >= 50.0) & P(lambda PH:PH == NEUTRO) & P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Revisar las mediciones realizadas.");
        

#rule 27
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo >= 45.0) & P(lambda PH:PH == NEUTRO) & P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Salinidad en el suelo")


#rule 28
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo <= 40.0) &  P(lambda arena:arena <= 40.0) & P(lambda arcilla:arcilla <= 40.0) & P(lambda PH:PH == NEUTRO) & P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        
#rule 29
@Rule(AnalisisDeSuelo(arcilla=P(lambda arcilla:arcilla >= 40.0) & P(lambda PH:PH == NEUTRO) & P(lambda CE:CE == BAJA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Baja mineralizacion de MO (Baja actvidad microbiologica)	");
        print("     |-(2) Baja difusion de Oxigeno y flujo de gases");    
        print("     |-(3) Limitaciones de movimiento de agua");
        
#rule 30
@Rule(AnalisisDeSuelo(arena=P( lambda arena:arena >= 50.0) & P(lambda PH:PH == NEUTRO) & P(lambda CE:CE == BAJA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Revisar las mediciones realizadas.");


#rule 31
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo >= 45.0) & P(lambda PH:PH == NEUTRO) & P(lambda CE:CE== BAJA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Coloraciones grises suelo (Glaizeado)");
        
#rule 32
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo <= 40.0) &  P(lambda arena:arena <= 40.0) & P(lambda arcilla:arcilla <= 40.0) & P(lambda PH:PH == NEUTRO) & P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        
#rule 33
@Rule(AnalisisDeSuelo(arcilla=P(lambda arcilla:arcilla >= 40.0) & P(lambda PH:PH == "LIGERAMENTE ACIDO") & P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Limitaciones de movimiento de agua");
        print("     |-(2) Baja difusion de Oxigeno y flujo de gases");    
        print("     |-(3) Baja mineralizacion de MO (Baja actvidad microbiologica");
        print("     |-(4) Acumulacion de iones alcalinoterreos");
        print("     |-(5) Alta saturación de calcio");
        print("     |-(6) Salinidad en el suelo");
        print("     |-(7) Baja disponibilidad de Fosforo (Precipitación)");
            
        
#rule 34
@Rule(AnalisisDeSuelo(arena=P(lambda arena:arena >= 50.0) & P(lambda PH:PH == "LIGERAMENTE ACIDO")& P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Revisar las mediciones realizadas.");
        
#rule 35
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo >= 45.0) & P(lambda PH:PH == "LIGERAMENTE ACIDO") & P(lambda CE:CE == ALTA)) )            
def metodo():
        print("\n\n\n");
        print("     |-(1) Contenido de Aluminio");
        print("     |-(2) Sulfatos altos");    
        print("     |-(3) Impedancia");
        
        
#rule 36
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo <= 40.0) &  P(lambda arena:arena <= 40.0) & P(lambda arcilla:arcilla <= 40.0) &  P(lambda PH:PH == "LIGERAMENTE ACIDO") & P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Baja disponibilidad de Fosforo (Precipitación)");
        print("     |-(2) Baja disponibilidad de Calcio");  
                       

#rule 37
@Rule(AnalisisDeSuelo(P(lambda arcilla:arcilla >= 40.0) & P(lambda PH:PH == "LIGERAMENTE ACIDO") & P(lambda CE:CE== BAJA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Limitaciones de movimiento de agua");
        print("     |-(2) Baja difusion de Oxigeno y flujo de gases");    
        print("     |-(3) Baja mineralizacion de MO (Baja actvidad microbiologica");
        print("     |-(4) Acumulacion de iones alcalinoterreos	");
        print("     |-(5) Baja disponibilidad de elementos menores ");       
        
        
#rule 38
@Rule(AnalisisDeSuelo( arena =P(lambda arena:arena >= 50.0) & P(lambda PH:PH == "LIGERAMENTE ACIDO") & P(lambda CE:CE == BAJA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Revisar las mediciones realizadas.");
        
#rule 39
@Rule(AnalisisDeSuelo( limo=P(lambda limo:limo >= 45.0) & P(lambda PH:PH == "LIGERAMENTE ACIDO") & P(lambda CE:CE== BAJA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Coloraciones grises suelo (Glaizeado)");
        print("     |-(2) Suelo Hidromorfico");    
        print("     |-(3) Limitaciones fisicas temporales");
        print("     |-(4) Baja difusion de Oxigeno y flujo de gases");
        print("     |-(5) Baja disponibilidad de elementos menores ");
        
        
#rule 40
@Rule(AnalisisDeSuelo( limo=P( lambda limo:limo <= 40.0) &  P(lambda arena:arena <= 40.0) & P(lambda arcilla:arcilla <= 40.0) & P(lambda PH:PH == "LIGERAMENTE ACIDO") & P(lambda CE:CE== ALTA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Baja disponibilidad de Fosforo (Precipitación)");
        print("     |-(2) Baja disponibilidad de Calcio");   
        print("     |-(3) Baja disponibilidad de elementos menores "); 
                     

#rule 41
@Rule(AnalisisDeSuelo(arcilla=P( lambda  arcilla:arcilla >= 40.0) & P(lambda PH:PH == ACIDO) & P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Limitaciones de movimiento de agua");
        print("     |-(2) Baja difusion de Oxigeno y flujo de gases");    
        print("     |-(3) Acumulacion de iones alcalinoterreos	");
        print("     |-(4) Salinidad en el suelo");
        print("     |-(5) Baja disponibilidad de Fosforo (Precipitación)");
        print("     |-(6) Baja disponibilidad de Calcio");
        print("     |-(7) Contenido de Aluminio");
        
           
        
#rule 42
@Rule(AnalisisDeSuelo(arena=P(lambda arena:arena >= 50.0) & P(lambda PH:PH == ACIDO) & P(lambda CE:CE == ALTA)) )            
def metodo():
        print("\n\n\n");
        print("     |-(1) Revisar las mediciones realizadas.");

#rule 43
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo >= 45.0) & P(lambda PH:PH ==  ACIDO) & P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Contenido de Aluminio");
        print("     |-(2) Sulfatos altos");    
        print("     |-(3) Impedancia");
        
#rule 44
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo <= 40.0) & P( lambda arena:arena <= 40.0) & P(lambda arcilla:arcilla <= 40.0) & P(lambda PH:PH == ACIDO) & P(lambda CE:CE == ALTA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Baja disponibilidad de Fosforo (Precipitación)");
        print("     |-(2) Baja disponibilidad de Calcio"); 
        
#rule 45
@Rule(AnalisisDeSuelo(arcilla=P(lambda arcilla:arcilla >= 40.0) & P(lambda PH:PH == ACIDO )& P(lambda CE:CE == BAJA)) )            
def metodo():
        print("\n\n\n");
        print("     |-(1) Limitaciones de movimiento de agua");
        print("     |-(2) Baja difusion de Oxigeno y flujo de gases");    
        print("     |-(3) Baja mineralizacion de MO (Baja actvidad microbiologica");
        print("     |-(4) Acumulacion de iones alcalinoterreos	");
        print("     |-(5) Baja disponibilidad de elementos menores ");

        
#rule 46
@Rule(AnalisisDeSuelo(arena=P(lambda arena:arena >= 50.0) & P(lambda PH:PH == ACIDO) & P(lambda CE:CE == BAJA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Revisar las mediciones realizadas.");
    
    
#rule 47
@Rule(AnalisisDeSuelo(limo=P(lambda limo:limo >= 45.0) & P(lambda PH:PH == ACIDO) & P(lambda CE:CE == BAJA)) )            
def metodo():
    
        print("\n\n\n");
        print("     |-(1) Coloraciones grises suelo (Glaizeado)");
        print("     |-(2) Suelo Hidromorfico");    
        print("     |-(3) Limitaciones fisicas temporales");
        print("     |-(4) Baja difusion de Oxigeno y flujo de gases");
        print("     |-(5) Baja disponibilidad de elementos menores ");
       